# meta_data를  data\normalized_data.csv 에 추가하기

## meta_data 읽기 

In [5]:
import polars as pl

df = pl.read_excel(
    r"C:\Users\Alice\OneDrive - 청주대학교\연구실 자료\연구실_노인균형프로토콜\데이터 정리-설문, 신체계측, 기타 데이터\perturb_inform.xlsm",
    sheet_name="meta"
)

# 첫 번째 열(subject)의 값들을 컬럼명으로 사용하여 전치
items = df["subject"].to_list()  # 이것이 새 컬럼명이 됨
subjects = df.columns[1:]         # 이것이 새 행 인덱스가 됨

transpose = (
    df
    .select(subjects)
    .transpose(include_header=True, header_name="subject", column_names=items)
)

print(transpose)

shape: (55, 87)
┌─────────┬──────┬──────┬───────┬───┬───────────────┬───────────────┬───────────────┬──────────────┐
│ subject ┆ 성별 ┆ 나이 ┆ 키    ┆ … ┆ 기초건강설문  ┆ 기초건강설문  ┆ 기초건강설문  ┆ 기초건강설문 │
│ ---     ┆ ---  ┆ ---  ┆ ---   ┆   ┆ 지_4          ┆ 지_5          ┆ 지_6          ┆ 지_특이사항  │
│ str     ┆ str  ┆ str  ┆ str   ┆   ┆ ---           ┆ ---           ┆ ---           ┆ ---          │
│         ┆      ┆      ┆       ┆   ┆ str           ┆ str           ┆ str           ┆ str          │
╞═════════╪══════╪══════╪═══════╪═══╪═══════════════╪═══════════════╪═══════════════╪══════════════╡
│ 김연옥  ┆ F    ┆ 72   ┆ 168.8 ┆ … ┆ null          ┆ null          ┆ null          ┆ null         │
│ 김윤자  ┆ F    ┆ 60   ┆ 152   ┆ … ┆ null          ┆ null          ┆ null          ┆ null         │
│ 전현숙  ┆ F    ┆ 67   ┆ 158.5 ┆ … ┆ null          ┆ null          ┆ null          ┆ null         │
│ 한영희  ┆ F    ┆ 73   ┆ 165   ┆ … ┆ null          ┆ null          ┆ null          ┆ null         │
│ 이순우  ┆ F    ┆ 78   ┆ 

In [6]:
# meta_subject_row: 행 = subject, 열 = item 구조
# "나이" 열 기준으로 young/old 그룹 생성

meta = transpose.with_columns(
    pl.when(pl.col("나이").cast(pl.Int32) < 30)
      .then(pl.lit("young"))
      .otherwise(pl.lit("old"))
      .alias("age_group")
)

print(meta)
print(meta.columns)

shape: (55, 88)
┌─────────┬──────┬──────┬───────┬───┬────────────────┬────────────────┬────────────────┬───────────┐
│ subject ┆ 성별 ┆ 나이 ┆ 키    ┆ … ┆ 기초건강설문지 ┆ 기초건강설문지 ┆ 기초건강설문지 ┆ age_group │
│ ---     ┆ ---  ┆ ---  ┆ ---   ┆   ┆ _5             ┆ _6             ┆ _특이사항      ┆ ---       │
│ str     ┆ str  ┆ str  ┆ str   ┆   ┆ ---            ┆ ---            ┆ ---            ┆ str       │
│         ┆      ┆      ┆       ┆   ┆ str            ┆ str            ┆ str            ┆           │
╞═════════╪══════╪══════╪═══════╪═══╪════════════════╪════════════════╪════════════════╪═══════════╡
│ 김연옥  ┆ F    ┆ 72   ┆ 168.8 ┆ … ┆ null           ┆ null           ┆ null           ┆ old       │
│ 김윤자  ┆ F    ┆ 60   ┆ 152   ┆ … ┆ null           ┆ null           ┆ null           ┆ old       │
│ 전현숙  ┆ F    ┆ 67   ┆ 158.5 ┆ … ┆ null           ┆ null           ┆ null           ┆ old       │
│ 한영희  ┆ F    ┆ 73   ┆ 165   ┆ … ┆ null           ┆ null           ┆ null           ┆ old       │
│ 이순우  ┆ F    ┆ 

## `meta` df 내 변수를 "data\normalized_data.csv"에 추가하기

연령변수(young/old) 생성 및 추가
주손 or 주발 변수 추가

In [7]:
normalized = pl.read_parquet(r"C:\Users\Alice\OneDrive - 청주대학교\근전도 분석 코드\shared_files\output\03_post_processed\normalized_data.parquet")

meta_select = meta.select(["subject", "age_group", "주손 or 주발"])
merged = normalized.join(meta_select, on="subject", how="left")

print(merged.head(5))
print(merged.columns)

# parquet 파일로 저장
merged.write_parquet(r"C:\Users\Alice\OneDrive - 청주대학교\근전도 분석 코드\aggregated_signal_viz\data\merged.parquet", compression="zstd", compression_level=10)
print("✓ 저장 완료")

shape: (5, 43)
┌────────────┬───────────┬────────────┬──────────┬───┬──────────┬──────────┬───────────┬───────────┐
│ MocapFrame ┆ MocapTime ┆ DeviceFram ┆ Fx       ┆ … ┆ EST      ┆ ESL      ┆ age_group ┆ 주손 or   │
│ ---        ┆ ---       ┆ e          ┆ ---      ┆   ┆ ---      ┆ ---      ┆ ---       ┆ 주발      │
│ i64        ┆ f64       ┆ ---        ┆ f64      ┆   ┆ f64      ┆ f64      ┆ str       ┆ ---       │
│            ┆           ┆ i64        ┆          ┆   ┆          ┆          ┆           ┆ str       │
╞════════════╪═══════════╪════════════╪══════════╪═══╪══════════╪══════════╪═══════════╪═══════════╡
│ 2981       ┆ 29.81     ┆ 0          ┆ 1.757342 ┆ … ┆ 0.064118 ┆ 0.041575 ┆ young     ┆ R         │
│ 2981       ┆ 29.81     ┆ 1          ┆ 1.783943 ┆ … ┆ 0.063004 ┆ 0.041727 ┆ young     ┆ R         │
│ 2981       ┆ 29.81     ┆ 2          ┆ 1.811277 ┆ … ┆ 0.060625 ┆ 0.041139 ┆ young     ┆ R         │
│ 2981       ┆ 29.81     ┆ 3          ┆ 1.839123 ┆ … ┆ 0.057156 ┆ 0.039855 ┆ you

In [8]:
print(merged["subject"].unique().to_list())  # subject 고유값 전체 출력

['김연옥', '신동석', '송석란', '김세준', '강비은', '이운복', '유병한', '이훈', '이은수', '임상우', '신향식', '임남순', '정혜진', '여윤숙', '연윤희', '김영자', '김윤자', '성효은', '노선자', '이지수', '문경서', '이순우', '이재유', '유재원', '안지연', '방주원', '정성윤', '박진우', '김우연', '한정란', '유도경', '정태웅', '방수진', '한영희', '박수빈', '이명희', '최지유', '조민석', '권유영', '이상민', '김병수', '오나영', '김종철', '김서하', '오인태', '가윤호', '김유민', '전현숙', '최보민', '이인섭', '김민정', '이정재']
